# MF2 TESTING

In [ ]:
import sandy
import pandas as pd
import io
pd.options.display.max_rows = 20

## LRU=0  (thorium 233 in jeff 3.3)

In [ ]:
tape = sandy.get_endf6_file("jeff_33", "xs", 902330)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])

In [ ]:
print (dic)

In [ ]:
tab = sandy.write_mf2(dic)

In [ ]:
print (tab)

## LRU = 1 

### LRF = 1 or LRF = 2 ( thorium 230 in jeff 3.3)

In [ ]:
tape = sandy.get_endf6_file("jeff_33", "xs", 902300)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])

In [ ]:
dic['NIS'][90230]['NER'][(1e-05, 251.0)]["L"]

In [ ]:
tab = sandy.write_mf2(dic)

In [ ]:
print (tab)

### LRF = 3  (Chlorine 37 in ENDF/B-VIII.0)

In [ ]:
tape = sandy.get_endf6_file("endfb_80", "xs", 170370)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])

In [ ]:
print (dic)

In [ ]:
tab = sandy.write_mf2(dic)

In [ ]:
print (tab)

### LRF = 7  (iron 54 in ENDF/B-VIII.0)

In [ ]:
tape = sandy.get_endf6_file("endfb_80", "xs", 260540)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])

In [ ]:
print (dic)

In [ ]:
tab2 = sandy.write_mf2(dic)

In [ ]:
print (tab)

## LRU = 2

### LFW = 0 and LRF = 1  (Iron 58 in ENDF/B-VIII.0)

In this part we only find isotopes that have multiple ranges of energy. Since the non resolved resonance parameters exist only for high energies. for this exemple LRU = 2 LRF = 1 is tested with LRU = 1 LRF = 3 in the lower energy ranges. 

In [ ]:
tape = sandy.get_endf6_file("endfb_80", "xs", 260580)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])

In [ ]:
print (dic['NIS'][26058]['NER'][(350000.0, 3000000.0)]["L"][3])

In [ ]:
tab = sandy.write_mf2(dic)

In [ ]:
print (tab)

### LRF = 2  (uranium 235 in Jeff 3.3)

here LRU = 2 LRF = 2 is tested with LRU = 1 LRF = 3 in the reading part we can choose to print only the  LRU = 2 LRF = 2 part by printing the dictionary that corespond to the high energy range. 

In [ ]:
tape = sandy.get_endf6_file("jeff_33", "xs", 922350)

In [ ]:
dic = sandy.read_mf2(tape,tape.mat[0])

In [ ]:
print (dic["NIS"][92235]["NER"][(2250.0, 46200.0)]['L'][0]["J"][3.0]['RES_PAR'][0])

In [ ]:
tab = sandy.write_mf2(dic)

In [ ]:
print (tab)

# Another  test

In [ ]:
def get_df_from_text(text, delimiter="?"):
    """
    Get the dataframe of a `endf6` file in a text. 
    Parameters
    ----------
    text : `str`
        Text with the data in `endf6` format.
    delimiter : `str`, optional
        Delimeter of the different lines. The default is "?".
    Returns
    -------
    `pd.DataFrame`
        Dataframe with the information of the text introduced as input.
    Examples
    -------
    XS:
        >>> endf6 = sandy.get_endf6_file("jeff_33", 'xs', 922350)
        >>> mt = 2
        >>> mat = 9228
        Obtain section information and replace blanks with zeros:
        >>> section = endf6._get_section_df(mat, 3, mt).apply(pd.to_numeric, errors='coerce').fillna(0)
        Get the same section using read and write functions:
        >>> read_xs = sandy.sections.mf3.read_mf3(endf6, mat, mt)
        >>> write_xs = sandy.sections.mf3.write_mf3(read_xs)
        >>> written_section = get_df_from_text(write_xs)
        Check the dataframes:
        >>> assert section.equals(written_section)
    """
    def foo(x):
        return sandy.shared.add_delimiter_every_n_characters(
            x[:66],
            11,
            delimiter=delimiter,
        )
    newtext = "\n".join(map(foo, text.splitlines())).replace('"', '*')
    df = pd.read_csv(
        io.StringIO(sandy.shared.add_exp_in_endf6_text(newtext)),
        delimiter=delimiter,
        na_filter=True,
        names=["C1", "C2", "L1", "L2", "N1", "N2"],
    )
    df = df.apply(pd.to_numeric, errors='coerce')
    return df.fillna(0)

In [ ]:
tape = sandy.get_endf6_file("jeff_33", "xs", 922350) #test LRU1,3 et LRU2,2

In [ ]:
section = tape._get_section_df(tape.mat[0], 2, 151).apply(pd.to_numeric, errors='coerce').fillna(0)

In [ ]:
read_xs = sandy.read_mf2(tape, tape.mat[0])

In [ ]:
write_xs = sandy.write_mf2(read_xs)

In [ ]:
written_section = get_df_from_text(write_xs)

In [ ]:
err = section == written_section

In [ ]:
err

In [ ]:
assert section.equals(written_section)